In [1]:
import pandas as pd
import numpy as np
from datetime import datetime, time as dtime, timedelta
import requests
import json
import os

import warnings

warnings.filterwarnings("ignore")
pd.set_option("display.max_columns", None)

In [31]:
def getOrders(start_date, end_date, manual_order, store_branch=None):
    query = """
    query GetOrders($offset: Float!, $limit: Float!, $isManualOrder: Boolean, $storeBranchId: Float, $unixTimeStart: Float, $unixTimeEnd: Float) {
      getOrders(offset: $offset, limit: $limit, isManualOrder: $isManualOrder, storeBranchId: $storeBranchId, unixTimeStart: $unixTimeStart, unixTimeEnd: $unixTimeEnd) {
        id
        orderNumber
        totalProduct
        totalQuantity
        subtotal
        discount
        cashback
        productCompliment
        transactionCompliment
        totalCompliment
        storeFee
        tax
        deliveryDiscount
        deliveryCost
        total
        status
        isManual
        isRecord
        isSettled
        isPromotionHasRelation
        isReadyOrder
        groupOrderRef
        timeCreated
        timeModified
        timeMaxProcess
        timeMaxProcessFinish
        timeProcessFinish
        timeMaxDelivery
        timeDelivered
        timeMaxFinished
        timeFinished
        storeBranchData {
            id
            name
            description
            extCode
            locationData {
                timeDifference
            }
        }
        employmentData {
            id
            employmentNumber
            employeeData {
                name
            }
        }
        transactionData {
            id
            status
            transactionNumber
            paymentDatas {
                id
                paymentMethodData {
                    name
                    id
                    code
                }
                amount
                fee
                total
                change
            }
        }
        orderProductDatas {
            id
            quantity
            subtotal
            discount
            cashback
            storeFee
            tax
            total
            customerNote
            status
            promotionDatas {
                value
                id
                promotionData {
                    id
                    title
                    description
                    detail {
                        usedKey
                    }
                }
            }
            complimentDatas {
                id
                value
                percentage
                note
            }
            productData {
                id
                barcode
                price
            }
        }
        invoiceData {
            invoiceNumber
        }
        refOrderReturnData {
            orderNumber
        }
        customerData {
            fullname
            id
            gender
        }
    }
}
  """

    variables = {
        "offset": 0,
        "limit": 50,
        "isManualOrder": manual_order,
        "unixTimeStart": int(
            (
                datetime.combine(start_date, dtime(0, 0, 0)) - timedelta(hours=7)
            ).timestamp()
        ),
        "unixTimeEnd": int(
            (
                datetime.combine(end_date, dtime(23, 59, 59)) - timedelta(hours=7)
            ).timestamp()
        ),
    }

    if store_branch is not None:
        variables["storeBranchId"] = store_branch

    return query, variables

In [32]:
# Env
graphql_endpoint = os.environ.get("GRAPHQL_END_POINT")

auth = {"Authorization": os.environ.get("GRAPHQL_MPR_AUTH")}

In [33]:
query, variables = getOrders(
    start_date=pd.to_datetime("2024-07-07"),
    end_date=pd.to_datetime("2024-07-07"),
    manual_order=False,
)

In [34]:
response = requests.post(
    url=graphql_endpoint, json={"query": query, "variables": variables}, headers=auth
)

In [35]:
data = response.json().get("data").get("getOrders")

In [51]:
df_order = pd.json_normalize(data=data)

In [56]:
df_order.head(2)

,id,orderNumber,totalProduct,totalQuantity,subtotal,discount,cashback,productCompliment,transactionCompliment,totalCompliment,storeFee,tax,deliveryDiscount,deliveryCost,total,status,isManual,isRecord,isSettled,isPromotionHasRelation,isReadyOrder,groupOrderRef,timeCreated,timeModified,timeMaxProcess,timeMaxProcessFinish,timeProcessFinish,timeMaxDelivery,timeDelivered,timeMaxFinished,timeFinished,orderProductDatas,refOrderReturnData,storeBranchData.id,storeBranchData.name,storeBranchData.description,storeBranchData.extCode,storeBranchData.locationData.timeDifference,employmentData.id,employmentData.employmentNumber,employmentData.employeeData.name,transactionData.id,transactionData.status,transactionData.transactionNumber,transactionData.paymentDatas,invoiceData.invoiceNumber,customerData.fullname,customerData.id,customerData.gender
0,172034588288257,OOFL070724103900023,2,2,252900,0,0,0,0,0,0,25063,0,0,252900,6,False,False,True,True,True,None,1720345882,1720360915,1720691479,1720345882,1720345882,1720345882,1720345882,1720345882,1720345882,"[{'id': 8421600, 'quantity': 1, 'subtotal': 24...",None,1039,C1026|SHOWROOM MZ GANDARIA CITY NEW,11313|CS2031|C1026,C1026,0,3348,122317657,Febri Selviani Tanjung,172034588264361,1,TRX240707165122020HD,"[{'id': 5424715, 'paymentMethodData': {'name':...",IN10390707241600023,Guest,999,NaN
1,172034588422930,OOFL07072464300006,1,1,429900,171960,0,0,0,0,0,0,0,0,257940,6,False,False,True,True,True,None,1720345884,1720363113,1720691482,1720345884,1720345884,1720345884,1720345884,1720345884,1720345884,"[{'id': 8421602, 'quantity': 1, 'subtotal': 42...",None,643,C0541|SHOWROOM MC NAGOYA HILL BATAM,10936|CS1407|C0541,C0541,0,3331,122317665,Fitri Rahayu,172034588440908,1,TRX240707165124372VR,"[{'id': 5424716, 'paymentMethodData': {'name':...",IN6430707241600006,Guest,999,NaN


In [94]:
# normalize transaction payment
pd.json_normalize(
    data=data,
    record_path=["transactionData", "paymentDatas"],
    meta=[["transactionData", "id"]],
    meta_prefix="transactionData.",
    errors="ignore",
)

,id,amount,fee,total,change,paymentMethodData.name,paymentMethodData.id,paymentMethodData.code,transactionData.transactionData.id
0,5424715,252900,0,252900,0,QRIS,3,QRISOFFLINE,172034588264361
1,5424716,257900,0,303000,45100,Cash,1,CASH,172034588440908
2,5424717,429840,0,429840,0,Debit Card,43,DEBIT,172034589227417
3,5424719,359800,0,359800,0,Credit Card,44,CREDIT,172034590293173
4,5424721,646820,0,646820,0,QRIS,3,QRISOFFLINE,172034591304294
5,5424722,299950,0,299950,0,QRIS,3,QRISOFFLINE,172034591735286
6,5424724,562800,0,562800,0,QRIS,3,QRISOFFLINE,172034594101596
7,5424725,609800,0,50000,0,External Gift Card,65,XGIFT,172034594226894
8,5424726,609800,0,559800,0,QRIS,3,QRISOFFLINE,172034594226894
9,5424727,179900,0,179900,0,Credit Card,44,CREDIT,172034594344132
